# Multi-period Scheduling Simulation

Multi-period economic dispatch (ED) and unit commitment (UC) are also available.

In this case, we will show a 24-hour ED simulation.

In [1]:
import ams

In [2]:
ams.config_logger(stream_level=20)

## Load Case

In [3]:
sp = ams.load(ams.get_case('5bus/pjm5bus_demo.xlsx'),
              setup=True,
              no_output=True,)

Parsing input file "/Users/jinningwang/work/ams/ams/cases/5bus/pjm5bus_demo.xlsx"...
Input file parsed in 0.0709 seconds.
Zero Line parameters detected, adjusted to default values: rate_b, rate_c.
All bus type are PQ, adjusted given load and generator connection status.
System set up in 0.0023 seconds.


## Zonal Design

The scheduling models in AMS have been developed with a zonal structure, which can be inspected in the model ``Area``.

In AMS, model ``Area`` represents Balancing Authority territory, and ``Zone`` refers to loss zone.
Typically, an ``Area`` can cover multiple ``Zone``.
For more details about the geographical definitions used in power systems, visit <https://ps-wiki.github.io/wiki/area-zone-region/>

**Note**: Since version 0.9.14, model ``Region`` is renamed to ``Zone`` for clarity.

In [4]:
sp.Area.as_df()

,idx,u,name
uid,,,
0,1,1.0,1
1,2,1.0,2
2,3,1.0,3


In device ``Bus``, the Param ``area`` indicates the area of the bus.
Correspondingly, the region of generator and load are determined by the bus they connected.

In [5]:
sp.Bus.as_df()

,idx,u,name,Vn,vmax,vmin,v0,a0,xcoord,ycoord,area,zone,owner,type
uid,,,,,,,,,,,,,,
0,0,1.0,A,230.0,1.1,0.9,1.0,0.0,0,0,1,Zone_A,None,2
1,1,1.0,B,230.0,1.1,0.9,1.0,0.0,0,0,1,Zone_B,None,2
2,2,1.0,C,230.0,1.1,0.9,1.0,0.0,0,0,2,Zone_C,None,2
3,3,1.0,D,230.0,1.1,0.9,1.0,0.0,0,0,2,Zone_D,None,3
4,4,1.0,E,230.0,1.1,0.9,1.0,0.0,0,0,3,Zone_E,None,2


## Multi-period Dispatch Base

In AMS, multi-period dispatch involves devices in group ``Horizon``.
This group is developed to provide time-series data for multi-period dispatch.

In [6]:
sp.Horizon.models

OrderedDict([('EDTSlot', EDTSlot (24 devices) at 0x334949400),
             ('UCTSlot', UCTSlot (24 devices) at 0x113888a70)])

We can get the idx of StaticGens.

In [7]:
sp.StaticGen.get_all_idxes()

['PV_1', 'PV_3', 'PV_5', 'PV_2', 'Slack_4']

In ``EDTSlot``, Param ``sd`` refers the load factors of each region in each time slot, and Param ``ug`` represents the generator commitment status in each time slot.

To be more specific, EDT1 has ``sd=0.793,0.793,0.793``, which means the load factor of each area is 0.793 in the first time slot, and 0.756 in the second time slot.

Next, EDT1 has ``ug=1,1,1,1,1``, and it means the commitment status of generator PV_1, PV_3, PV_5, PV_2, and Slack_4 are all online.

In [8]:
sp.EDTSlot.as_df()

,idx,u,name,sd,ug
uid,,,,,
0,EDT1,1.0,EDT1,"0.793,0.793,0.793","1,1,1,1,1"
1,EDT2,1.0,EDT2,"0.756,0.756,0.756","1,1,1,1,1"
2,EDT3,1.0,EDT3,"0.723,0.723,0.723","1,1,1,1,1"
3,EDT4,1.0,EDT4,"0.708,0.708,0.708","1,1,1,1,1"
4,EDT5,1.0,EDT5,"0.7,0.7,0.7","1,1,1,1,1"
5,EDT6,1.0,EDT6,"0.706,0.706,0.706","1,1,1,1,1"
6,EDT7,1.0,EDT7,"0.75,0.75,0.75","1,1,1,1,1"
7,EDT8,1.0,EDT8,"0.802,0.802,0.802","1,1,1,1,1"
8,EDT9,1.0,EDT9,"0.828,0.828,0.828","1,1,1,1,1"


## Solve and Result

In [9]:
sp.ED.init()

Building system matrices
Parsing OModel for <ED>
Evaluating OModel for <ED>
Finalizing OModel for <ED>
<ED> initialized in 0.0421 seconds.


<ED> RParam <c2> should have all non-negative values.


True

In [10]:
sp.ED.run(solver='CLARABEL')

<ED> solved as optimal in 0.0312 seconds, converged in 14 iterations with CLARABEL.


True

All decision variables are collected in the dict ``vars``.

In [11]:
sp.ED.vars

OrderedDict([('pg', Var: StaticGen.pg),
             ('vBus', Var: Bus.vBus),
             ('aBus', Var: Bus.aBus),
             ('pru', Var: StaticGen.pru),
             ('prd', Var: StaticGen.prd),
             ('prs', Var: StaticGen.prs)])

As we can see, the generator output ``pg`` is a 2D array, and the first dimension is the generator index, and the second dimension is the time slot.

In [12]:
sp.ED.pg.v.round(4)

array([[0.2   , 0.2   , 0.2   , 0.2   , 0.2   , 0.2   , 0.2   , 0.2   ,
        0.2   , 0.2   , 0.2   , 0.2   , 0.2   , 0.2   , 0.2   , 0.2   ,
        0.2   , 0.2   , 0.2   , 0.2   , 0.2   , 0.2   , 0.2   , 0.2   ],
       [0.5   , 0.5   , 0.5   , 0.5   , 0.5   , 0.5   , 0.5   , 0.5   ,
        0.5751, 0.6908, 0.8064, 0.9271, 1.0327, 1.1735, 1.2891, 1.3746,
        1.44  , 1.44  , 1.3947, 1.2187, 1.088 , 0.9623, 0.6807, 0.5   ],
       [0.6   , 0.6   , 0.6   , 0.6   , 0.6   , 0.6   , 0.6   , 0.6   ,
        0.6   , 0.6   , 0.6   , 0.6   , 0.6   , 0.6   , 0.6   , 0.6   ,
        0.6   , 0.6   , 0.6   , 0.6   , 0.6   , 0.6   , 0.6   , 0.6   ],
       [4.8289, 4.8258, 4.8231, 4.8218, 4.8211, 4.8217, 4.8253, 4.8297,
        4.9049, 5.0192, 5.1336, 5.2529, 5.3573, 5.4965, 5.6109, 5.6954,
        5.76  , 5.76  , 5.7153, 5.5413, 5.412 , 5.2877, 5.0093, 4.8282],
       [1.8011, 1.4342, 1.1069, 0.9582, 0.8789, 0.9383, 1.3747, 1.8903,
        2.    , 2.    , 2.    , 2.    , 2.    , 2.    , 2.  

Partial results can be accessed with desired time slot.
In the retrieved result, the first dimension is the generator index, and the second dimension is the time slot.

In [13]:
sp.ED.get(src='pg', attr='v', idx='PV_1', horizon=['EDT1'])

np.float64(0.20000000298691178)

Or, get multiple variables in mutliple time slots.

In [14]:
sp.ED.get(src='pg', attr='v', idx=['PV_1', 'PV_3'],
          horizon=['EDT1', 'EDT2', 'EDT3']).round(4)

array([[0.2, 0.2, 0.2],
       [0.5, 0.5, 0.5]])